In [1]:
# extract different view of data
# view two, node2vec
setting = "n2v"

viewTwoFilesDir = "../Data/vectors/"+setting+"/data=Meta-alg=N2V-l2=1.0-n2v_p=0.85-iteration=100-no_self_predict=1-idx=0.emb"
viewTwoVectors = []

with open(viewTwoFilesDir, 'r', encoding = 'utf8') as f:
    for line in f:
        read_data = line.split(" ")
        if(len(read_data[0])<=8):
            paper_Vectors = read_data
            viewTwoVectors.append(paper_Vectors)
f.close()
viewTwoVectors = viewTwoVectors[1:]
print("Total vector records:",len(viewTwoVectors))
print(viewTwoVectors[0])

Total vector records: 8602916
['22516865', '0.0109272', '0.126011', '0.186979', '0.0496719', '0.0373553', '0.0458918', '-0.119893', '0.217118', '0.0524591', '0.237477', '0.191269', '-0.0277055', '0.0290957', '-0.0366833', '0.118964', '0.0654807', '-0.0335345', '-0.0900123', '0.128621', '0.0561669', '-0.087823', '-0.0882296', '0.0740289', '0.082104', '0.0269581', '-0.0346502', '0.0153376', '0.104666', '0.0908716', '-0.085694', '-0.111344', '0.0787209', '-0.17003', '-0.103366', '-0.0832094', '-0.210496', '0.153037', '-0.0342884', '0.0698413', '-0.0719641', '-0.0535707', '0.172399', '0.106226', '-0.0593672', '-0.0348048', '-0.0863189', '-0.0801566', '-0.0665761', '0.0673258', '0.0306541', '-0.0896316', '-0.00800971', '-0.174798', '-0.0252528', '0.0098563', '0.0230368', '0.0282268', '-0.0366493', '-0.131323', '0.0318188', '-0.00778704', '-0.0608064', '-0.0860078', '0.215632', '0.0209927', '-0.0953191', '-0.191736', '-0.0741615', '0.151972', '-0.0522046', '-0.11081', '0.134878', '0.090797',

In [2]:
import re
import os

# collect data
fileDir = "../Data/filteredSameNameAuthor/filter=30/"
fileList = os.listdir(fileDir)
fileList.sort()
print(fileList)

['chung-may yang.txt', 'chung-may yang0.txt', 'chung-may yang1.txt', 'david g lloyd.txt', 'david g lloyd0.txt', 'david g lloyd1.txt', 'jeong hwan kim.txt', 'jeong hwan kim0.txt', 'jeong hwan kim1.txt', 'kevin m. ryan.txt', 'kevin m. ryan0.txt', 'kevin m. ryan1.txt', 'lei wang.txt', 'lei wang0.txt', 'lei wang1.txt', 'michael wagner.txt', 'michael wagner0.txt', 'michael wagner1.txt']


In [3]:
# remove author(positive sample) from other(negative sample)
import random
def extractNegativeSample(positiveSample, allSample):
    negativeSample = [x for x in allSample if x not in positiveSample]
    print("Total negative sample size:", len(negativeSample))
    return negativeSample

In [4]:
# collect class vectors
import pandas as pd
import numpy as np

def extractVectors(author_pids, NegativeSample_pid, allPaperVectors):
    # extract class one vectors
    author_features = []
    for pid in author_pids:
         for paper_Vectors in allPaperVectors:
            if(paper_Vectors[0] == pid):
                author_features.append(paper_Vectors)
    print("Positive sample size: ", len(author_features))
    classOne = pd.DataFrame(author_features)
    classOne["label"] = 0
    # extract class two vectors
    other_features = []
    for pid in NegativeSample_pid:
        for paper_Vectors in allPaperVectors:
            if(paper_Vectors[0] == pid):
                other_features.append(paper_Vectors)
    print("Negative sample size: ", len(other_features))
    classTwo = pd.DataFrame(other_features)
    classTwo["label"] = 1
    return classOne, classTwo

In [16]:
# combine data from different class get all data
def combineClassesData(classOne,classTwo):
    combinedData = pd.concat([classOne, classTwo])
    combinedData = combinedData.sample(frac=1).reset_index(drop=True)
    # take the paper id out
    paperID = combinedData[0]
    # split data and label
    data = combinedData.drop([0,'label'], axis=1)
    label = combinedData['label']
    print("Total sample size and shape: ",data.shape)
    return data, label, paperID

In [8]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import (precision_score, recall_score,f1_score,accuracy_score)
# cross validation
def k_fold_cv(data, label, classifier, k=10):
    kf = KFold(n_splits=k, shuffle=False)
    allTrueLabel = []
    allPredLabel = []
    for counter,(train_index, test_index) in enumerate(kf.split(data)):
        # print("TRAIN:", train_index, " \n TEST:", test_index)
        # split train and test
        data_train, data_test = data.iloc[train_index], data.iloc[test_index]
        label_train, test_true_label = label.iloc[train_index], label.iloc[test_index]
        # fit data to svm
        classifier.fit(data_train, label_train)
        # get predicted label
        label_pred = classifier.predict(data_test)
        allTrueLabel.extend(test_true_label)
        allPredLabel.extend(label_pred)
#         # get predict proba
#         proba = classifier.predict_proba(data_test)
        # find out which sample cause the issue
        print("Pred: ",label_pred)
        print("True: ", test_true_label.values.tolist())
        print("Mislabeled sample: ",end='')
        for i in range(len(test_true_label)):
            if(label_pred[i]!=test_true_label[test_index[i]]):
                print(paperID[test_index[i]]+",",end='')
        print()
        # print("True positive: {tp}, False positive: {fp}, False negative: {fn}, True negative: {tn}"
        # .format(tp=round_tp, fp=round_fp, fn=round_fn, tn=round_tn))

    accuracy = accuracy_score(allTrueLabel, allPredLabel)
    f1 = f1_score(allTrueLabel, allPredLabel,average='binary')
    precision = precision_score(allTrueLabel, allPredLabel)
    recall = recall_score(allTrueLabel, allPredLabel)
    tn,fp,fn,tp = metrics.confusion_matrix(allTrueLabel, allPredLabel).ravel()
    
    print("Classifier: ",classifier)
    print(metrics.classification_report(allTrueLabel, allPredLabel))
    print(metrics.confusion_matrix(allTrueLabel, allPredLabel).ravel())
    print("Accuracy: ",accuracy)
    print("F1: ", f1)
    print("Precision: ", precision)
    print("Recall: ", recall)
    
    return accuracy, f1, precision, recall, tn, fp, fn, tp
    # return ppv, npv, specificity, sensitivity, accuracy, f1proba = linear_svc.predict_proba(allDatas)

In [9]:
# hard code to read the file one by one
# store the features for classification
author_pids = []
other_pids = []
# author as positive sample, other as all samples
with open(fileDir+"lei wang0.txt", 'r', encoding = 'utf8') as f:
    for line in f:
        author_pids.extend(line.strip().split(" "))

with open(fileDir+"lei wang.txt", 'r', encoding = 'utf8') as f:
    for line in f:
        other_pids.extend(line.strip().split(" "))
        
print(author_pids)
print(other_pids[0])

['27321135', '27078635', '26996321', '26301538', '25464845', '24615769', '25010185', '24449339', '25044429', '23913257', '24019075', '23322720', '24290358', '24139041', '22554080', '21956564', '22662873', '21805546', '21732687', '21681861', '21751313', '21926996', '21545173', '20872394', '20385807', '19318213', '19764747', '19668857', '19165727', '18426210', '18556466', '17603477', '17406421', '16689635', '16956756', '15840835', '15599909', '15556995', '15378068', '12580587', '12779328', '12518054', '14576413', '12537491', '12515477', '11866580', '12154230', '12148987', '12244330', '12120285', '12203503', '11564556', '11313494']
27321135


In [10]:
# extract negative Sample
NegativeSample_pid = extractNegativeSample(author_pids, other_pids)
print("Choicen negative sample ", len(NegativeSample_pid))

Total negative sample size: 64
Choicen negative sample  64


In [12]:
classOne, classTwo = extractVectors(author_pids,NegativeSample_pid,viewTwoVectors)
print(classOne.shape)
print(classTwo.shape)

Positive sample size:  50
Negative sample size:  62
(50, 102)
(62, 102)


In [17]:
data, label, paperID = combineClassesData(classOne, classTwo)
print(data)

Total sample size and shape:  (112, 100)
            1           2           3          4           5            6    \
0      0.312339   0.0694004    0.632244  0.0651315   -0.489079    -0.139653   
1      0.522446  -0.0980524       0.454  0.0218141   -0.639211    -0.081809   
2    -0.0280236    0.176684    0.226024   0.374915   -0.209283    0.0981133   
3      0.368911    0.201126    0.354669   0.356884   -0.542716    -0.368921   
4     0.0096327    0.268802  -0.0147734   0.228448   -0.415773     0.105026   
5      0.319688    0.256781    0.594806   0.241398   -0.771016    -0.329713   
6      0.313397    0.101994    0.766564   0.433176   -0.706534    -0.478016   
7     -0.295066    0.865667    0.236162   0.776292   -0.341878     0.277269   
8      0.351753  -0.0163507    0.638449   0.288624   -0.541191     -0.28179   
9      0.753287   -0.045984     0.68357   0.123793   -0.814707   -0.0590562   
10     0.246118     0.19095    0.597295   0.276298    -0.53077    -0.323592   
11      0.3

In [19]:
from sklearn import svm
linear_svc = svm.SVC(kernel='linear', class_weight='balanced', probability=True,cache_size=4000)
accuracy, f1, precision, recall, tn, fp, fn, tp= k_fold_cv(data, label, linear_svc,10)


Pred:  [1 1 0 1 0 1 1 0 1 1 1 1]
True:  [1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1]
Mislabeled sample: 
Pred:  [0 1 0 0 1 1 0 1 1 1 0 0]
True:  [0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0]
Mislabeled sample: 
Pred:  [0 1 0 0 0 1 0 1 0 0 0]
True:  [0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0]
Mislabeled sample: 
Pred:  [1 1 1 0 1 0 0 0 0 1 1]
True:  [1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1]
Mislabeled sample: 
Pred:  [1 0 0 1 1 1 0 0 0 1 0]
True:  [1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0]
Mislabeled sample: 
Pred:  [1 1 1 1 0 1 1 0 0 1 1]
True:  [1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1]
Mislabeled sample: 
Pred:  [0 1 1 0 0 0 1 1 1 0 1]
True:  [0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1]
Mislabeled sample: 
Pred:  [1 1 1 1 0 1 0 1 1 1 1]
True:  [1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]
Mislabeled sample: 
Pred:  [0 1 1 0 0 1 0 0 0 1 0]
True:  [0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0]
Mislabeled sample: 
Pred:  [0 1 1 1 1 0 1 1 0 0 0]
True:  [0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0]
Mislabeled sample: 
Classifier:  SVC(C=1.0, cache_size=4000, class_weight='balanced', coef